# Assignment 1, 8-Puzzle Programming

### Your homework must be implemented in this Notebook file. 
### You can add as many cells as you want. However, you are not allowed to touch the code below the line "=============".
### You need to implement the three (four for grads) searching functions and the print result functions.
### For the searching functions, feel free to customize the return data types and parameter lists as long as the function name is as required.


In [350]:
import numpy as np
import time
GOAL_STATE = [0,1,2,3,4,5,6,7,8]
GOAL_CORDINATES = {0:(0,0),1:(0,1),2:(0,2),3:(1,0),4:(1,1),5:(1,2),6:(2,0),7:(2,1),8:(2,2)}
flag = True

In [351]:
class Node:
    def __init__(self,state,parent,cost):
        self.state = state
        self.parent = parent
        self.cost = cost # g(n) or cost/depth of node in tree
        self.heuristic_cost = 0 #h(n)
        self.step_cost = 0 # g(n)+h(n)
        #Nodes to store next moves
        self.move_up =None
        self.move_down =None
        self.move_right= None
        self.move_left = None

In [352]:
def find_blank_tile(state):
    zero_index = [i for i in np.where(state==0)]
    return zero_index[0]

def find_possible_moves(node):
    next_possible_moves = []
    state = node.state
       
    #try moving tile up
    if is_legal_move(state=state,move ='N'):
        state_up = try_move(state,move='N')
        new_up_node = Node(state = state_up,parent = node,cost =node.cost+1)
        node.move_up = new_up_node
        next_possible_moves.append(new_up_node)
    
    #try moving tile down
    if is_legal_move(state= state,move='S'):
        state_down = try_move(state,move='S')
        new_down_node = Node(state = state_down,parent =node,cost =node.cost+1)
        node.move_down = new_down_node
        next_possible_moves.append(new_down_node)
                
    #try moving tile right
    if is_legal_move(state=state,move='E'):
        state_right = try_move(state,move='E')
        new_right_node = Node(state = state_right, parent = node,cost = node.cost+1)
        node.move_right = new_right_node
        next_possible_moves.append(new_right_node)
        
    #try moving tile left
    if is_legal_move(state = state,move='W'):
        state_left = try_move(state,move='W')
        new_left_node = Node(state = state_left, parent = node,cost = node.cost+1)
        node.move_left = new_left_node
        next_possible_moves.append(new_left_node)   
          
    return next_possible_moves

def h_misplaced_cost(state):
    h_cost = np.sum(state != GOAL_STATE)-1
    if h_cost >0:
        return h_cost
    else:
        return 0
    
def h_manhattan_cost(state):
    h_cost = 0
    for i in range(3):
        for j in range(3):
            if state[i*3+j] != 0:
                man_dis = sum(abs(a-b) for a,b in zip((i,j),GOAL_CORDINATES[state[i*3+j]]))
                h_cost = h_cost + man_dis
    return h_cost

def is_goal_state(state):
    goal = np.array(GOAL_STATE,dtype= np.int8)
    if(np.array_equal(goal,state)):
        return True
    else:
        return False

def is_legal_move(state,move):
    zero_index = find_blank_tile(state)
    index = zero_index[0]
    if move =='E':
        if index % 3 == 2: #Invalid right move
            return False    
    elif move=='W':
        if index%3 ==0: #Invalid left move
            return False
    elif move == 'N': #Invalid move up
        if index<3:
            return False 
    elif move == 'S':  # Invalid move down
        if index > 5:
            return False
    
    return True

def is_state_visited(state,visited):
    if hash(tuple(state)) in visited:
        return True
    else:
        return False
       
def is_state_explored(state,explored):
    return next((True for element in explored if element is state),False)

def swap(state,ind1,ind2):
        temp = state[ind1]
        state[ind1] = state[ind2]
        state[ind2] = temp
        return state
    
def try_move(state,move):
    index = find_blank_tile(state)
    next_state = np.copy(state)
    if move == 'N':
        new_state = swap(next_state,index,index-3)
    
    elif move == 'S':
        new_state = swap(next_state,index,index+3)
        
    elif move == 'E': 
        next_state = swap(next_state,index,index+1)
            
    elif move == 'W':
        next_state = swap(next_state,index,index-1)
            
    else:
        return None
        
    return next_state

In [353]:
#implementation of function "A Star Misplaced"
def a_star_misplaced(node):

    start_time = time.time()
    state = node.state
    frontier = [(node,0)]#queue to store unvisited states and thier heiuristic cost
    visited = set() # to keep track of visited states
    explored = set()#to keep track of states added to frontier but not yet visited

    while frontier:
        frontier = sorted(frontier, key =lambda x: x[1]) # Frontier are sorted based on 
                                                        # path_cost (g(n)+h(n))
        curr_node = frontier.pop(0)[0]
        curr_state = curr_node.state
        visited.add(hash(tuple(curr_state)))
        
        if(is_goal_state(curr_state)):
            end_time = time.time()
            step_count = print_result_ASM(node =curr_node)
            return step_count,end_time-start_time
        else:
            next_possible_moves = find_possible_moves(curr_node)
            for move in next_possible_moves:
                if not hash(tuple(move.state)) in visited:
                    #if not hash(tuple(move.state)) in explored:
                    h_cost = h_misplaced_cost(move.state)
                    move.heuristic_cost = h_cost
                    move.step_cost = move.cost + h_cost                   
                    frontier.append((move,move.step_cost))
                        #explored.add(hash(tuple(move.state)))
                    
def print_result_ASM(node):
    global flag
    
    state_tracker = [node.state]
    state_cost =[node.step_cost]
    state_level =[node.cost]
    count = 1
    while node.parent:
        node =node.parent
        state_tracker.append(node.state)
        state_cost.append(node.step_cost)
        state_level.append(node.cost)
        count = count+1   
    if flag:
        print("Moves to reach solution via A STAR MISPLACED TILES:")
        flag = False
        print(np.array(state_tracker.pop()).reshape(3,3))
        #print("Step cost: ",state_cost.pop()," level cost: ",state_level.pop())
        while state_tracker:
            print('to')
            print(np.array(state_tracker.pop()).reshape(3,3))
            #print("Step cost: ",state_cost.pop()," level cost: ",state_level.pop())
    return count

In [354]:
#implementation of function "A Star Manhattan Distance"
def a_star_manhattan_distance(node):
    start_time = time.time()
    state = node.state
    frontier = [(node,0)]#queue to store unvisited states and thier heiuristic cost
    visited = set() # to keep track of visited states
    explored = set()#to keep track of states added to frontier but not yet visited

    while frontier:
        frontier = sorted(frontier, key =lambda x: x[1]) # Frontier are sorted based on 
                                                         # path_cost (g(n)+h(n))
        curr_node = frontier.pop(0)[0]
        curr_state = curr_node.state
        
        visited.add(hash(tuple(curr_state)))
        
        if(is_goal_state(curr_state)):
            end_time = time.time()
            step_count = print_result_ASMD(node =curr_node)
            return step_count,end_time-start_time
        else:
            next_possible_moves = find_possible_moves(curr_node)
            for move in next_possible_moves:
                if not hash(tuple(move.state)) in visited:
                    if not hash(tuple(move.state)) in explored:
                        h_cost = h_manhattan_cost(move.state)
                        move.heuristic_cost = h_cost
                        move.step_cost = move.cost + h_cost                   
                        frontier.append((move,move.step_cost))
                        explored.add(hash(tuple(move.state)))
                    
def print_result_ASMD(node):
    global flag
    
    state_tracker = [node.state]
    state_cost =[node.step_cost]
    state_level =[node.cost]
    count = 1
    while node.parent:
        node =node.parent
        state_tracker.append(node.state)
        state_cost.append(node.step_cost)
        state_level.append(node.cost)
        count = count+1   
    if flag:
        print("\n Moves to reach solution via A STAR MANHATTAN DISTANCE:")
        flag = False
        print(np.array(state_tracker.pop()).reshape(3,3))
        #print("Step cost: ",state_cost.pop()," level cost: ",state_level.pop())
        while state_tracker:
            print('to')
            print(np.array(state_tracker.pop()).reshape(3,3))
            #print("Step cost: ",state_cost.pop()," level cost: ",state_level.pop())
    return count

In [355]:
def read_input_file():
    file = open("Input8PuzzleCases.txt","r")
    inputlist=[]
    index =0
    for line in file:
        newline = line.strip("\n").split(",")
        inputlist.append(np.array(newline,dtype=np.int8))
        index+=1        
    file.close()
    return inputlist

In [356]:
#implementation of function "print_result(result)"
def print_result(result):
    global flag
    inputlist =read_input_file()
    print("File read")
    
    #Solution using A STAR MISPLACED TILES
    flag = True
    asm_time_list = []
    asm_stepcount_list = []
    #lineNum = 1 #used to print only first solution
    for i in range (100):
        line = np.array(inputlist[i],dtype= np.int8)
        root_node = Node(state = line,parent = None,cost = 0)
        #print("STATE: ",line)
        step_count,time_taken = a_star_misplaced(root_node)
        asm_time_list.append(time_taken)
        #print("TIME TAKEN: ",time_taken)
        asm_stepcount_list.append(step_count)
        #lineNum = lineNum+1
        
    #Solution using A STAR MANHATTAN DISTANCE
    flag = True
    asmd_time_list = []
    asmd_stepcount_list = []
    for i in range (100):
        input = np.array(inputlist[i],dtype= np.int8)
        root_node = Node(state = input,parent = None,cost = 0)
        #print("STATE: ",input)
        step_count_asmd,time_taken_asmd = a_star_manhattan_distance(root_node)
        asmd_time_list.append(time_taken_asmd)
        #print("TIME TAKEN: ",time_taken_asmd)
        asmd_stepcount_list.append(step_count_asmd)
        
        
    #Print final results
    print('\n\n\t\t\t Average_Time \t\t Average_Steps')
    print('\t ASM \t\t ', sum(asm_time_list)/100, ' \t \t', sum(asm_stepcount_list)/100)
    print('\t ASMD \t\t ',sum(asmd_time_list)/100, ' \t \t', sum(asmd_stepcount_list)/100)

if __name__ == "__main__":
    print_result(None)

File read
Moves to reach solution via A STAR MISPLACED TILES:
[[8 7 5]
 [4 1 2]
 [3 0 6]]
to
[[8 7 5]
 [4 0 2]
 [3 1 6]]
to
[[8 7 5]
 [4 2 0]
 [3 1 6]]
to
[[8 7 0]
 [4 2 5]
 [3 1 6]]
to
[[8 0 7]
 [4 2 5]
 [3 1 6]]
to
[[8 2 7]
 [4 0 5]
 [3 1 6]]
to
[[8 2 7]
 [4 1 5]
 [3 0 6]]
to
[[8 2 7]
 [4 1 5]
 [3 6 0]]
to
[[8 2 7]
 [4 1 0]
 [3 6 5]]
to
[[8 2 0]
 [4 1 7]
 [3 6 5]]
to
[[8 0 2]
 [4 1 7]
 [3 6 5]]
to
[[0 8 2]
 [4 1 7]
 [3 6 5]]
to
[[4 8 2]
 [0 1 7]
 [3 6 5]]
to
[[4 8 2]
 [1 0 7]
 [3 6 5]]
to
[[4 0 2]
 [1 8 7]
 [3 6 5]]
to
[[0 4 2]
 [1 8 7]
 [3 6 5]]
to
[[1 4 2]
 [0 8 7]
 [3 6 5]]
to
[[1 4 2]
 [3 8 7]
 [0 6 5]]
to
[[1 4 2]
 [3 8 7]
 [6 0 5]]
to
[[1 4 2]
 [3 0 7]
 [6 8 5]]
to
[[1 4 2]
 [3 7 0]
 [6 8 5]]
to
[[1 4 2]
 [3 7 5]
 [6 8 0]]
to
[[1 4 2]
 [3 7 5]
 [6 0 8]]
to
[[1 4 2]
 [3 0 5]
 [6 7 8]]
to
[[1 0 2]
 [3 4 5]
 [6 7 8]]
to
[[0 1 2]
 [3 4 5]
 [6 7 8]]

 Moves to reach solution via A STAR MANHATTAN DISTANCE:
[[8 7 5]
 [4 1 2]
 [3 0 6]]
to
[[8 7 5]
 [4 0 2]
 [3 1 6]]
to
[[8 7 5]
 [4 2 0

You can insert as many cells as you want above
You are not Allowed to modify the code below this line.
===============================

In [ ]:
#you need to implement print_result function to print out the result according to the required format
print_result(None)


# The output format should be as follows. You only need to give one sample solution as an example.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Solution of the first Scenario:
#### X X X
#### X X X
#### X X X
#### to
#### X X X
#### X X X
#### X X X
#### to
#### X X X
#### X X X
#### X X X
#### to
#### X X X
#### X X X
#### X X X
#### to
#### .
#### .
#### .
#### 0 1 2
#### 3 4 5
#### 6 7 8

                Average_Steps    Average_Time      
 IDS
 
 BFS

